In [1]:
import os, sys
os.environ["HF_HOME"]='/home/jovyan/.cache/huggingface'
os.environ["HUGGINGFACE_HUB_CACHE"]='/home/jovyan/.cache/huggingface/hub' 
os.environ["TRANSFORMERS_CACHE"]='/home/jovyan/.cache/huggingface/hub'
os.environ["HF_DATASETS_CACHE"]='/home/jovyan/.cache/huggingface/datasets'

from transformers import AutoModelForSeq2SeqLM, AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig
import pandas as pd
import torch

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.8/site-packages/transformers/utils/hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
prompts = list(map(lambda x: x.strip() if x.strip()[-1] == '?' else x.strip() + '?', pd.read_csv('local_prompt.csv', sep='\t')['prompt'].values))

FileNotFoundError: [Errno 2] No such file or directory: 'local_prompt.csv'

OSError: Incorrect path_or_model_id: './output_v2/cendol-mt5-xl/checkpoint-100000'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

# mT5 V2

In [19]:
tokenizer = AutoTokenizer.from_pretrained('./output_v2/cendol-mt5-xxl-merged')
for model_path in [
    './output_v2/cendol-mt5-small/checkpoint-100000',
    './output_v2/cendol-mt5-base/checkpoint-100000',
    './output_v2/cendol-mt5-large/checkpoint-100000',
    './output_v2/cendol-mt5-xl/checkpoint-100000',
    './output_v2/cendol-mt5-xxl-merged/'
]:
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path, torch_dtype=torch.float16).cuda()
    model_name = model_path.split('/')[-2]
    print(f'===== {model_name} =====')
    responses = []
    for prompt in prompts:
        responses.append(
            tokenizer.decode(
                model.generate(tokenizer(prompt, return_tensors='pt')['input_ids'].cuda(), top_p=0.9, top_k=50, min_length=0, max_length=256, do_sample=True)[0]
            )
        )
        print(f'prompt: {prompt}')
        print(f'response: {responses[-1]}')
    pd.DataFrame({'prompts': prompts, 'responses': responses}).to_csv(f'./gen_save/{model_name}_v2.csv', index=False)

===== cendol-mt5-small =====
prompt: Apa itu STNK?
response: <pad> STNK adalah sebuah perusahaan berbasis dari perusahaan-perusahaan ini yang aktif dan didirikan pada tahun 1995, menjadikannya salah satu perusahaan yang didirikan pada tahun 1996, dan menjadi mitra aktif pada tahun 1997 setelah berperan sebagai seorang aktris dan penulis lagu Amerika Serikat. Pada tahun 2003, STNK secara komersial menjadi perusahaan yang membentuk sebuah pusat pendidikan di Jakarta, dengan jumlah keseluruhan dan mencapai peringkat ganda sebesar 340.000. STNK berbasis pada tahun 2012, dengan jumlah penduduk sebesar 5.975 jiwa dan memproduksi 2 juta jiwa per tahun. Tahun 2005, STNK memiliki peringkat ganda sebesar 10.6 juta jiwa. Di Indonesia, STNK memiliki kemampuan untuk meningkatkan pertumbuhan suara dan memperoleh kontribusi menjadi satu bintang utama dengan mewakilinya menjadi satu bintang. Pada tahun 2007, STNK membangun sebuah perusahaan berbasis di Singapura, yang memiliki kapasitas yang tinggi. S

Loading checkpoint shards: 100%|██████████| 6/6 [00:50<00:00,  8.35s/it]


===== cendol-mt5-xxl-merged =====
prompt: Apa itu STNK?
response: <pad> STNK adalah protokol yang dibuat oleh PNS untuk menyatakan berbagai jenis data, dan untuk mengatur berbagai jenis transaksi. STNK merupakan protokol transaksi multilevel yang menyediakan transaksi menggunakan nomor akun yang dimiliki PNS. Ketika berisi data, STNK juga dapat mengakses berbagai jenis aplikasi lain, seperti media sosial dan aplikasi perusahaan lainnya, seperti LinkedIn, YouTube, dan Instagram.</s>
prompt: Gimana caranya perpanjang STNK?
response: <pad> Ini gan alamatnya, Cendol bukan ahli hukum, jadi tidak bisa bantu juga untuk masalah seperti ini. Jika ada masalah terkait STNK, bisa coba hubungi nomor hotline berikut ini: - Surat Dinas Pendidikan Nasional</s>
prompt: Tutorial perpanjang STNK?
response: <pad> Perpanjang STNK itu apaan ya gan? Cendol gaptek nih ga paham juga.</s>
prompt: Di mana perpanjang STNK?
response: <pad> Untuk memperpanjang STNK bisa ke kantor polisi yang berada di provinsi Lamp

# mT5 V1

In [3]:
tokenizer = AutoTokenizer.from_pretrained('./output_v2/cendol-mt5-xxl-merged')
for model_path in [
    './output/cendol-mt5-small/checkpoint-140000',
    './output/cendol-mt5-base/checkpoint-140000',
    './output/cendol-mt5-large/checkpoint-140000',
    './output/cendol-mt5-xl/checkpoint-140000',
    'indonlp/cendol-mt5-xxl-merged'
]:
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path, torch_dtype=torch.float16).cuda()
    model_name = model_path.split('/')[-2]
    print(f'===== {model_name} =====')
    responses = []
    for prompt in prompts:
        responses.append(
            tokenizer.decode(
                model.generate(tokenizer(prompt, return_tensors='pt')['input_ids'].cuda(), top_p=0.9, top_k=50, min_length=0, max_length=256, do_sample=True)[0]
            )
        )
        print(f'prompt: {prompt}')
        print(f'response: {responses[-1]}')
    print()
    pd.DataFrame({'prompts': prompts, 'responses': responses}).to_csv(f'./gen_save/{model_name}_v1.csv', index=False)

===== cendol-mt5-small =====


/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


prompt: Apa itu STNK?
response: <pad> TNS atau nilai berapa nilai STNK?</s>
prompt: Gimana caranya perpanjang STNK?
response: <pad> Bagaimana cara saya memperpanjang STNK?</s>
prompt: Tutorial perpanjang STNK?
response: <pad> Cara Bagaimana Penting Ruang Kredit Tbk?</s>
prompt: Di mana perpanjang STNK?
response: <pad> Dimana perpanjangan STNK?</s>
prompt: Cara mengurus STNK yang sudah tidak berlaku?
response: <pad> Berapa banyak surat yang tertulis pada surat kabar?</s>
prompt: Cara mengurus KTP hilang?
response: <pad> Cara mengurus KTP?</s>
prompt: KTP gue ilang coy?
response: <pad> KTP ku hilang coy?</s>
prompt: Kapan harus bayar pajak?
response: <pad> Apa yang harus menjadi bayar pajak?</s>
prompt: Gimana cara ngurus NPWP?
response: <pad> Apa cara terbaik untuk mengoreksi NPWP?</s>
prompt: Cara pembuatan NPWP?
response: <pad> Cara pembuatan NPWP?</s>
prompt: Apa itu BPJS?
response: <pad> Apa itu BPJS?</s>
prompt: Gimana caranya daftar BPJS?
response: <pad> Bagaimana cara membuat daf

OSError: indonlp/cendol-mt5-xxl is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [3]:
tokenizer = AutoTokenizer.from_pretrained('./output_v2/cendol-mt5-xxl-merged')
for model_path in [
#     './output/cendol-mt5-small/checkpoint-140000',
#     './output/cendol-mt5-base/checkpoint-140000',
#     './output/cendol-mt5-large/checkpoint-140000',
#     './output/cendol-mt5-xl/checkpoint-140000',
    'indonlp/cendol-mt5-xxl-merged'
]:
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
    model_name = model_path.split('/')[-2]
    print(f'===== {model_name} =====')
    responses = []
    for prompt in prompts:
        responses.append(
            tokenizer.decode(
                model.generate(tokenizer(prompt, return_tensors='pt')['input_ids'], top_p=0.9, top_k=50, min_length=0, max_length=256, do_sample=True)[0]
            )
        )
        print(f'prompt: {prompt}')
        print(f'response: {responses[-1]}')
    print()
    pd.DataFrame({'prompts': prompts, 'responses': responses}).to_csv(f'./gen_save/{model_name}_v1.csv', index=False)

Loading checkpoint shards: 100%|██████████| 6/6 [00:49<00:00,  8.18s/it]


===== indonlp =====
prompt: Apa itu STNK?
response: <pad> Apa itu STNK?</s>
prompt: Gimana caranya perpanjang STNK?
response: <pad> Langkah apa saja yang harus dilakukan untuk memperpanjang STNK?</s>
prompt: Tutorial perpanjang STNK?
response: <pad> Tutorial Perpanjang STNK</s>
prompt: Di mana perpanjang STNK?
response: <pad> Di mana perpanjang STNK?</s>
prompt: Cara mengurus STNK yang sudah tidak berlaku?
response: <pad> Apa yang harus dilakukan jika STNK sudah tidak berlaku?</s>
prompt: Cara mengurus KTP hilang?
response: <pad> Bagaimana cara mengurus KTP hilang?</s>
prompt: KTP gue ilang coy?
response: <pad> KTP aku hilang coy</s>
prompt: Kapan harus bayar pajak?
response: <pad> Kapan harus membayar pajak?</s>
prompt: Gimana cara ngurus NPWP?
response: <pad> Bagaimana cara mengurus NPWP?</s>
prompt: Cara pembuatan NPWP?
response: <pad> Cara pembuatan NPWP</s>
prompt: Apa itu BPJS?
response: <pad> Apa itu BPJS?</s>
prompt: Gimana caranya daftar BPJS?
response: <pad> Bagaimana cara da

# LLaMA V2

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('./output_v2/cendol-mt5-xxl-merged')
for model_path in [
    'indonlp/cendol-llama2-7b-merged-chat',
    'indonlp/cendol-llama2-13b-merged-chat'
]:
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path, torch_dtype=torch.float16).cuda()
    model_name = model_path.split('/')[-1]
    print(f'===== {model_name} =====')
    responses = []
    for prompt in prompts:
        responses.append(
            tokenizer.decode(
                model.generate(tokenizer(prompt, return_tensors='pt')['input_ids'].cuda(), top_p=0.9, top_k=50, min_length=0, max_length=256, do_sample=True)[0]
            )
        )
        print(f'prompt: {prompt}')
        print(f'response: {responses[-1]}')
    pd.DataFrame({'prompts': prompts, 'responses': responses}).to_csv(f'./gen_save/{model_name}_v2.csv', index=False)

# LLaMA V1

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('./output_v2/cendol-mt5-xxl-merged')
for model_path in [
    'indonlp/cendol-llama2-7b-merged',
    'indonlp/cendol-llama2-13b-merged'
]:
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path, torch_dtype=torch.float16).cuda()
    model_name = model_path.split('/')[-1]
    print(f'===== {model_name} =====')
    responses = []
    for prompt in prompts:
        responses.append(
            tokenizer.decode(
                model.generate(tokenizer(prompt, return_tensors='pt')['input_ids'].cuda(), top_p=0.9, top_k=50, min_length=0, max_length=256, do_sample=True)[0]
            )
        )
        print(f'prompt: {prompt}')
        print(f'response: {responses[-1]}')
    pd.DataFrame({'prompts': prompts, 'responses': responses}).to_csv(f'./gen_save/{model_name}_v1.csv', index=False)